In [1]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

# やりたいこと
+ サブクエリのネストにならずに複数のjoin条件をSQL実装する

## 3つのテーブルをacct_nbr軸に結合する
+ acct_nbrとfoo_qty(合計値)、bar_qty(合計値)を繋げる
+ sql puzzle #74

In [3]:
%%sql
drop table if exists Foo, Bar, Accounts;
CREATE TABLE Accounts
 (acct_nbr INTEGER NOT NULL PRIMARY KEY);

CREATE TABLE Foo
 (acct_nbr INTEGER NOT NULL
    REFERENCES Accounts(acct_nbr),
  foo_qty INTEGER NOT NULL);

CREATE TABLE Bar
(acct_nbr INTEGER NOT NULL
    REFERENCES Accounts(acct_nbr),
 bar_qty INTEGER NOT NULL);

INSERT INTO Accounts VALUES (1);
INSERT INTO Accounts VALUES (2);
INSERT INTO Accounts VALUES (3);
INSERT INTO Accounts VALUES (4);

INSERT INTO Foo VALUES (1, 10);
INSERT INTO Foo VALUES (2, 20);
INSERT INTO Foo VALUES (2, 40);
INSERT INTO Foo VALUES (3, 80);

INSERT INTO Bar VALUES (2, 160);
INSERT INTO Bar VALUES (3, 320);
INSERT INTO Bar VALUES (3, 640);
INSERT INTO Bar VALUES (3, 1);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [4]:
%%sql
-- こちらは普通にやった時の回答
select tmp.acct_nbr, tot_foo_qty, sum(bar_qty) as tot_bar_qty
from (
    select a.acct_nbr, sum(foo_qty)
    from Accounts as a
    left join Foo as f
    on a.acct_nbr = f.acct_nbr
    group by a.acct_nbr
) as tmp(acct_nbr, tot_foo_qty)
left join Bar as b
on tmp.acct_nbr = b.acct_nbr
group by tmp.acct_nbr, tot_foo_qty
order by acct_nbr

*  postgresql://padawan:***@db:5432/dsdojo_db
4 rows affected.


acct_nbr,tot_foo_qty,tot_bar_qty
1,10,None
2,60,160
3,80,961
4,None,None


In [24]:
%%sql
select acct_nbr, sum(foo_qty), sum(bar_qty)
from  (
    (
        select a.acct_nbr, foo_qty, bar_qty
        from Accounts as a
        left join (select acct_nbr, foo_qty, 0 as bar_qty from foo) as f
        on a.acct_nbr = f.acct_nbr
    )
    union all
    (
        select a.acct_nbr, foo_qty, bar_qty
        from Accounts as a
        left join (select acct_nbr, 0 as foo_qty, bar_qty from bar) as b
        on a.acct_nbr = b.acct_nbr
    )
) as tmp
group by acct_nbr
order by acct_nbr

*  postgresql://padawan:***@db:5432/dsdojo_db
4 rows affected.


acct_nbr,sum,sum_1
1,10,0
2,60,160
3,80,961
4,None,None


## 3つのテーブルをitem_nbr軸に結合する
+ estimated_amt(合計値), actual_amt(合計値)とする
+ sql puzzle #41

In [5]:
%%sql
drop table if exists Items, Actuals, Estimates;
CREATE TABLE Items
(item_nbr INTEGER,
 item_descr CHAR(10));

CREATE TABLE Actuals
(item_nbr   INTEGER, 
 actual_amt DECIMAL(5,2),
 check_nbr  CHAR(4));

CREATE TABLE Estimates
(item_nbr      INTEGER, 
 estimated_amt DECIMAL(5,2));

INSERT INTO Items VALUES(10, 'Item 10');
INSERT INTO Items VALUES(20, 'Item 20');
INSERT INTO Items VALUES(30, 'Item 30');
INSERT INTO Items VALUES(40, 'Item 40');
INSERT INTO Items VALUES(50, 'item 50');

INSERT INTO Actuals VALUES(10, 300.00, '1111');
INSERT INTO Actuals VALUES(20, 325.00, '2222');
INSERT INTO Actuals VALUES(20, 100.00, '3333');
INSERT INTO Actuals VALUES(30, 525.00, '1111');

INSERT INTO Estimates VALUES(10, 300.00);
INSERT INTO Estimates VALUES(10, 50.00);
INSERT INTO Estimates VALUES(20, 325.00);
INSERT INTO Estimates VALUES(20, 110.00);
INSERT INTO Estimates VALUES(40, 25.00);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [17]:
%%sql
-- 先ほどと同様にfromの中をunion allで繋げてみる
select
    item_nbr,
    max(item_descr),
    sum(est) as est_sum,
    sum(act) as act_sum,
    case
        when count(check_nbr) >= 2 then 'mixed' else max(check_nbr)
    end as check_nbr
from (
    (
        select item_nbr, item_descr, 0 as est, 0 as act, NULL as check_nbr
        from Items
    )
    union all
    (
        select item_nbr, null as item_descr, 0 as est, actual_amt as act, check_nbr as check_nbr
        from Actuals
    )
    union all
    (
        select item_nbr, null as item_descr, estimated_amt as est, 0 as act, NULL as check_nbr
        from Estimates
    )
) as tmp
group by item_nbr
having sum(est) <> 0 or sum(act) <> 0
order by item_nbr

*  postgresql://padawan:***@db:5432/dsdojo_db
4 rows affected.


item_nbr,max,est_sum,act_sum,check_nbr
10,Item 10,350.00,300.00,1111
20,Item 20,435.00,425.00,mixed
30,Item 30,0,525.00,1111
40,Item 40,25.00,0,None


## 
+ --
+ sql puzzle #51

In [19]:
%%sql
drop table if exists Budgeted, Actual;
create table Budgeted (
    task integer not null primary key,
    category integer not null,
    est_cost decimal(8,2) not null
);

create table actual (
    voucher integer not null primary key,
    task integer not null references Budgeted(task),
    act_cost decimal(8,2) not null
);

INSERT INTO Budgeted VALUES(1, 9100, 100.00);
INSERT INTO Budgeted VALUES(2, 9100,  15.00);
INSERT INTO Budgeted VALUES(3, 9100,   6.00);
INSERT INTO Budgeted VALUES(4, 9200,   8.00);
INSERT INTO Budgeted VALUES(5, 9200,  11.00);

INSERT INTO Actual VALUES(1,  1,  10.00);
INSERT INTO Actual VALUES(2,  1,  20.00);
INSERT INTO Actual VALUES(3,  1,  15.00);
INSERT INTO Actual VALUES(4,  2,  32.00);
INSERT INTO Actual VALUES(5,  4,  8.00);
INSERT INTO Actual VALUES(6,  5,  3.00);
INSERT INTO Actual VALUES(7,  5,  4.00);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [20]:
%%sql
-- 普通の解き方
select category, sum(est_cost), sum(sum_act)
from Budgeted as b
left join (
    select task, sum(act_cost)
    from Actual
    group by task
) as a(task, sum_act)
on b.task = a.task
group by category

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


category,sum,sum_1
9100,121.00,77.00
9200,19.00,15.00


In [26]:
%%sql
select b.category,
    sum(est_cost),
    sum(act_cost)
from (
    (
        select task, est_cost, 0 as act_cost
        from Budgeted
    )
    union all
    (
        select task, 0 as est_cost, act_cost as act_cost
        from Actual
    )
) as tmp
inner join (select task, category from Budgeted) as b
on tmp.task = b.task
group by b.category

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


category,sum,sum_1
9100,121.00,77.00
9200,19.00,15.00
